# 3.1 Tickers
The goal is to solve the problems of the Polygon ticker lists in the introduction. Before we do that we will download the ticker list for all days from Polygon and store them into the map <code>tickers</code>.

In [22]:
###
from polygon.rest import RESTClient
from datetime import datetime, date, time, timedelta
from pytz import timezone
from functools import lru_cache
from times import get_market_dates, first_trading_date_after, last_trading_date_before, first_trading_date_after_equal, last_trading_date_before_equal
import os
import pytz
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import mplfinance as mpf

POLYGON_DATA_PATH = "../data/polygon/"

START_DATE = date(2019, 6, 1)
END_DATE = date(2024, 3, 1)

with open(POLYGON_DATA_PATH + "secret.txt") as f:
    KEY = next(f).strip()

client = RESTClient(api_key=KEY)

First, I will create a function to download the ticker for a specific date.

In [23]:
###
def download_tickers(day):
    """Retrieve the ticker list for a specific date

    Args:
        day (Date): the Date for which to download the ticker list

    Returns:
        DataFrame: the ticker list
    """
    
    date_iso = day.isoformat()

    ticker_list_iterator_active = client.list_tickers(type="CS", date=date_iso, active=True, market='stocks', limit=1000)
    ticker_list_iterator_delisted = client.list_tickers(type="CS", date=date_iso, active=False, market='stocks', limit=1000)
    ticker_list_iterator_active_adr = client.list_tickers(type="ADRC", date=date_iso, active=True, market='stocks', limit=1000)
    ticker_list_iterator_delisted_adr = client.list_tickers(type="ADRC", date=date_iso, active=False, market='stocks', limit=1000)
    tickers_active = pd.DataFrame(ticker_list_iterator_active)
    tickers_delisted = pd.DataFrame(ticker_list_iterator_delisted)
    tickers_active_adr = pd.DataFrame(ticker_list_iterator_active_adr)
    tickers_delisted_adr = pd.DataFrame(ticker_list_iterator_delisted_adr)

    tickers_all = pd.concat([tickers_active, tickers_delisted, tickers_active_adr, tickers_delisted_adr])
    tickers_all.sort_values(by = "ticker", inplace=True)
    tickers_all.reset_index(inplace=True, drop=True)
    return tickers_all[['ticker', 'name', 'active', 'delisted_utc', 'last_updated_utc', 'cik', 'composite_figi', 'type']]

Then all ticker lists are downloaded and stored in the <code>raw/tickers/</code> map. But only the one that we need if we already have some.

In [24]:
###
# Get a list of what we already have
files = os.listdir(POLYGON_DATA_PATH + 'raw/tickers')
available_dates = [date.fromisoformat(file.replace(".csv", "")) for file in files]

trading_dates = get_market_dates()
for day in trading_dates:
    # Only download what we do not have
    if day >= START_DATE and day <= END_DATE and day not in available_dates:
        tickers = download_tickers(day)
        tickers.to_csv(POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv")
        print(f"Downloaded tickers for {day.isoformat()}")

A random ticker list:

In [25]:
pd.read_csv(POLYGON_DATA_PATH + f"raw/tickers/2022-06-09.csv", index_col=0).head(3)

,ticker,name,active,delisted_utc,last_updated_utc,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,NaN,2022-06-14T00:00:00Z,1090872.0,BBG000C2V3D6,CS
1,AA,"Alcoa, Inc.",False,2016-11-01T00:00:00Z,2016-11-01T00:00:00Z,4281.0,NaN,CS
2,AA,ALCOA INC,False,2016-10-07T00:00:00Z,2016-10-07T00:00:00Z,4281.0,NaN,CS


We observe that the <code>last_updated_utc</code> does not match the date of the ticker list. For example for "A", this date is *after* 2022-06-09. So this value is not point-in-time. Nevertheless, this value is useless for us because we only use the data to determine the delisting date. Neither do we need <code>delisted_utc</code> for the same reason.

Later when we do have data, we will create a new column <code>start_data</code> and <code>end_data</code> which gives the start and end dates from the available data.

# 3.2 Building the tickers loop
Now we can finally create our ticker list, which includes all tickers. The process involves looping over all Polygon ticker lists and updating our own one. First some notation: T is our ticker list that we iteratively update using Polygons ticker list. P(i) is the Polygon ticker list from day *i*. 

1. On day 1, our ticker list is the same as the one from Polygon, but with some extra columns. We create a column <code>start_date</code> which is day 1 and <code>end_date</code> with is empty. We are only interested in stocks that were active on that day.
2. For all *i = 2 ... n* days, for the active stocks:
    * **Delistings**: The stocks that are in T but not in P(i) are the stocks that are removed by Polygon (e.g. FB). For these tickers we set the <code>end_date</code> in T to day *i-1*. 
    * **New listings**: The stocks that are in P(i) but not in T are the new listings. We will append the new stock to T and set the start_date to day *i*.
    * **Everything else**: The stocks that are both in P(i) and T are the stocks that 'continue their listings'. We do nothing.

Two tickers are the 'same' if all fields except <code>last_updated_utc</code> or <code>delisted_utc</code> are the same.

For testing, we will start with 2022-06-08 and update to 2022-06-09. Both FB and META should then be included with the correct start and end dates. The start and end date of FB should be 2022-06-08 and the start date of META should be 2022-06-09. The end date of META should be empty.

In [26]:
day_1 = date(2022, 6, 8)
day_2 = date(2022, 6, 9)

our_tickers = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_1.isoformat()}.csv",
    index_col=0,
)
our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
our_tickers = our_tickers[our_tickers["active"] == True]
our_tickers.reset_index(inplace=True, drop=True)

our_tickers["start_date"] = day_1
our_tickers["end_date"] = pd.NaT

tickers_day_2 = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/{day_2.isoformat()}.csv",
    index_col=0,
)
tickers_day_2 = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]]
tickers_day_2 = tickers_day_2[tickers_day_2["active"] == True]
tickers_day_2.reset_index(inplace=True, drop=True)

In [27]:
our_tickers.head(2)

,ticker,name,active,cik,composite_figi,type,start_date,end_date
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS,2022-06-08,NaT
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS,2022-06-08,NaT


In [28]:
tickers_day_2.head(2)

,ticker,name,active,cik,composite_figi,type
0,A,Agilent Technologies Inc.,True,1090872.0,BBG000C2V3D6,CS
1,AA,Alcoa Corporation,True,1675149.0,BBG00B3T3HD3,CS


Preliminary check for duplicates

In [29]:
# Preliminary check: no duplicates
if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().any():
    raise Exception("There are duplicates!")

if tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().any():
    raise Exception("There are duplicates!")

We will first get the delisting and new listings. (Nothing has to be done with the kept listings).

In [30]:
# DELISTINGS: Get tickers that are in T but not in P(2). This is actually not straightforward (https://stackoverflow.com/questions/28901683/pandas-get-rows-which-are-not-in-other-dataframe). We need to get the rows in tickers_day_2 that are not in our_tickers. We will use the merge function but specifying indicator=True and use a left merge (tickers_day_2 left merge to our_tickers). What gets returned is a dataframe with the flags "left_only", "right_only" and "both". If the indicator is "left_only", it means that it existed in only in the left DataFrame (our_tickers). This is exactly what we need. 
indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_delisted = indicator_delisted["_merge"] # Only get the indicator

delisted_tickers = our_tickers[indicator_delisted == "left_only"] # Only get the delisted tickers

# NEW LISTINGS: Swap the DataFrames
indicator_new = tickers_day_2[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                   how='left', indicator=True)
indicator_new = indicator_new["_merge"]
new_tickers = tickers_day_2[indicator_new == "left_only"]

# KEPT LISTINGS
current_tickers = our_tickers[indicator_delisted == "both"]
# current_tickers = tickers_day_2[indicator_new == "both"] # It does not matter which one we choose

In [31]:
print(len(delisted_tickers))
delisted_tickers.head(2)

6


,ticker,name,active,cik,composite_figi,type,start_date,end_date
1144,CERN,Cerner Corp,True,804753.0,BBG000BFDLV8,CS,2022-06-08,NaT
1799,DYNS,Dynamics Special Purpose Corp. Class A Common ...,True,1854270.0,BBG010WX7ZB3,CS,2022-06-08,NaT


In [32]:
print(len(new_tickers))
new_tickers.head(2)

5


,ticker,name,active,cik,composite_figi,type
2435,GLAQ,Globis Acquisition Corp. common stock,True,1823383.0,NaN,CS
2725,HOUS,Anywhere Real Estate Inc.,True,1398987.0,BBG000QN4GY3,CS


In [33]:
len(current_tickers)

6275

Then we will process the delistings and listings.

In [34]:
# DELISTINGS: register delisting date and set to inactive.
our_tickers.loc[indicator_delisted == "left_only", "end_date"] = day_1 #Not day_2!
our_tickers.loc[indicator_delisted == "left_only", "active"] = False

our_tickers[our_tickers["ticker"] == "FB"]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2080,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08


In [35]:
# NEW LISTINGS: append the new tickers and register start date
print(len(our_tickers))
print(len(new_tickers))

our_tickers = pd.concat([our_tickers, new_tickers])
our_tickers.reset_index(inplace=True, drop=True)
our_tickers['start_date'].fillna(value=day_2, inplace=True)

print(len(our_tickers))
our_tickers[our_tickers["ticker"] == "META"]

6281
5
6286


,ticker,name,active,cik,composite_figi,type,start_date,end_date
6283,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,NaN


Some final checks and setting <code>end_date</code> for the active listings at END_DATE.

In [36]:
if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
    raise Exception("There are missing values.")

# After all is done, set the end_date for active stocks to the new day. This is only done after all iterations. 
our_tickers["end_date"].fillna(value=day_2, inplace=True)

The result is correct. FB is included with the correct <code>end_date</code>. Then META starts with the correct <code>start_date</code>.

In [37]:
our_tickers[our_tickers['ticker'].isin(['FB', 'META'])]

,ticker,name,active,cik,composite_figi,type,start_date,end_date
2080,FB,"Meta Platforms, Inc. Class A Common Stock",False,1326801.0,BBG000MM2P62,CS,2022-06-08,2022-06-08
6283,META,"Meta Platforms, Inc. Class A Common Stock",True,1326801.0,BBG000MM2P62,CS,2022-06-09,2022-06-09


Further down the line, I got an error that I wouldn't have gotten if I checked for duplicates in the (original) ticker lists. In the original ticker lists for the active tickers there should never be duplicates. Because each ticker can only have one listing. However the polygon ticker lists do have these:

In [38]:
our_tickers = pd.read_csv(
    POLYGON_DATA_PATH + f"raw/tickers/2024-03-01.csv",
    index_col=0,
)
our_tickers = our_tickers[our_tickers["active"] == True]
our_tickers.sort_values(['last_updated_utc'])
our_tickers.reset_index(inplace=True, drop=True)

duplicated = our_tickers[our_tickers["ticker"].duplicated(keep=False)]
duplicated[duplicated["ticker"].duplicated(keep=False)]

,ticker,name,active,delisted_utc,last_updated_utc,cik,composite_figi,type
939,CADE,Cadence Bancorporation,True,NaN,2019-01-03T00:00:00Z,1614184.0,BBG00BZSZZD5,CS
940,CADE,Cadence Bank,True,NaN,2024-03-28T00:00:00Z,1299939.0,BBG000D3MJP5,CS
3421,MRT,"MedEquities Realty Trust, Inc.",True,NaN,2019-01-03T00:00:00Z,1616314.0,NaN,CS
3422,MRT,"Marti Technologies, Inc.",True,NaN,2024-03-28T00:00:00Z,1852767.0,BBG00ZXBPJV8,CS


We need to make sure to remove these. We will simply keep the last one as that is the most recent one.

In [39]:
indices_to_remove = duplicated["ticker"].duplicated(keep='last')
list(indices_to_remove[indices_to_remove].index)

[939, 3421]

In [40]:
our_tickers.drop(list(indices_to_remove[indices_to_remove].index), inplace=True)
our_tickers.reset_index(drop=True, inplace=True)

# 3.3 The tickers loop
Putting it all in a loop gives the following code. We save the results to <code>tickers_v1.csv</code>.

In [ ]:
market_days = get_market_dates()

first_trading_date_after_equal_start_date = first_trading_date_after_equal(START_DATE)
last_trading_date_before_equal_end_date = last_trading_date_before_equal(END_DATE)

for day in market_days:
    if day == first_trading_date_after_equal_start_date:
        # At the start, our ticker list is the same as polygon.
        our_tickers = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{first_trading_date_after_equal_start_date.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        ) #There is a stock named 'NA'. We have to avoid pandas treating it as a N/A value.
        our_tickers = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        our_tickers = our_tickers[our_tickers["active"]]

        # Sometimes a cik can be empty. This prevents the merge function from working.
        our_tickers['cik'] = our_tickers['cik'].replace('', np.nan)
        our_tickers['cik'] = our_tickers['cik'].astype(float)

        our_tickers.reset_index(inplace=True, drop=True)
    
        # Initialize tickers_all
        our_tickers["start_date"] = first_trading_date_after_equal_start_date
        our_tickers["end_date"] = pd.NaT

    elif day > first_trading_date_after_equal_start_date and day <= last_trading_date_before_equal_end_date:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        # Sometimes a cik can be empty. This prevents the merge function from working.
        tickers_day_i['cik'] = tickers_day_i['cik'].replace('', np.nan)
        tickers_day_i['cik'] = tickers_day_i['cik'].astype(float)

        tickers_day_i.sort_values(['last_updated_utc'], inplace=True)

        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"]]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Remove duplicates
        duplicated = tickers_day_i[tickers_day_i["ticker"].duplicated(keep=False)]
        indices_to_remove = duplicated["ticker"].duplicated(keep='last')
        tickers_day_i.drop(list(indices_to_remove[indices_to_remove].index), inplace=True)
        tickers_day_i.reset_index(drop=True, inplace=True)

        # Preliminary check: no duplicates
        if our_tickers.duplicated().any():
            raise Exception("There are duplicates!")

        if tickers_day_i.duplicated().any():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])
        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)

        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        # print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == last_trading_date_before_equal_end_date:
            our_tickers["end_date"].fillna(value=last_trading_date_before_equal_end_date, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].to_csv("../data/tickers_v1.csv")

We also create a function to retrieve the ticker list.

In [43]:
def get_tickers(v=5):
    """
    Retrieve the ticker list. Default is 5.
    """
    tickers = pd.read_csv(
        f"../data/tickers_v{v}.csv",
        parse_dates=["start_date", "end_date"],
        index_col=0,
        keep_default_na=False,
        na_values=["#N/A","#N/AN/A","#NA","-1.#IND","-1.#QNAN","-NaN","-nan","1.#IND","1.#QNAN","<NA>","N/A","NULL","NaN","None","n/a","nan","null",],
    )
    tickers["start_date"] = pd.to_datetime(tickers["start_date"]).dt.date
    tickers["end_date"] = pd.to_datetime(tickers["end_date"]).dt.date

    # This will only be applied in future notebooks.
    if tickers.columns.isin(["start_data", "end_data"]).any():
        tickers["start_data"] = pd.to_datetime(tickers["start_data"]).dt.date
        tickers["end_data"] = pd.to_datetime(tickers["end_data"]).dt.date

    return tickers

In [44]:
tickers_v1 = get_tickers(1)
tickers_v1[tickers_v1["ticker"] == "FB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
5243,FB,"Facebook, Inc. Class A",False,2019-06-03,2021-10-29,CS,1326801.0,BBG000MM2P62
5244,FB,"Meta Platforms, Inc. Class A Common Stock",False,2021-11-01,2022-06-08,CS,1326801.0,BBG000MM2P62


In [45]:
tickers_v1[tickers_v1["ticker"] == "META"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
8750,META,"Meta Platforms, Inc. Class A Common Stock",True,2022-06-09,2024-03-01,CS,1326801.0,BBG000MM2P62


In [46]:
print(len(tickers_v1))

15266


# 3.4 Checks

1. Are SPACs handled correctly? We should expect that when they IPO a company, that they get delisted. Then one day after the delisting the new-born company should be listed. We will take a look at VFS. On 2023-8-15 it was IPO'd by the SPAC named BSAQ. So we should expect the delisting date of BSAQ to be 2023-8-14.


In [47]:
tickers_v1[tickers_v1["ticker"] == "VFS"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
14336,VFS,VinFast Auto Ltd. Ordinary Shares,True,2023-08-15,2024-03-01,CS,1913510.0,


In [48]:
tickers_v1[tickers_v1["ticker"] == "BSAQ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
2128,BSAQ,Black Spade Acquisition Co,False,2021-09-07,2023-08-14,CS,1851908.0,


2. Let's check SVB which went bankrupt and HTZ which went from OTC to listed.

In [49]:
tickers_v1[tickers_v1["ticker"] == "SIVB"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
12505,SIVB,SVB Financial Group,False,2019-06-03,2023-03-27,CS,719739.0,BBG000BT0CM2


In [50]:
tickers_v1[tickers_v1["ticker"] == "HTZ"]

,ticker,name,active,start_date,end_date,type,cik,composite_figi
6914,HTZ,"Hertz Global Holdings, Inc.",False,2019-06-03,2020-10-29,CS,1657853.0,BBG00D5SHJH6
6915,HTZ,"Hertz Global Holdings, Inc Common Stock",True,2021-11-09,2024-03-01,CS,1657853.0,BBG011N57109


3. Sometimes tickers are re-used (e.g. META, but since it was an ETF it will not show up in our ticker list). Let's see if that has happened in our ticker list.

In [51]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
print(len(duplicated))
duplicated.head()

3462
9585


,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,Agilent Technologies,False,2019-06-03,2019-08-16,CS,1090872.0,BBG000C2V3D6
1,A,Agilent Technologies Inc.,False,2019-08-19,2022-02-07,CS,1090872.0,BBG000C2V3D6
2,A,Agilent Technologies Inc.,False,2022-02-08,2022-02-08,CS,1090872.0,BBG000BWQYZ5
3,A,Agilent Technologies Inc.,True,2022-02-09,2024-03-01,CS,1090872.0,BBG000C2V3D6
6,AAC,"AAC Holdings, Inc.",False,2019-06-03,2019-10-25,CS,1606180.0,BBG00K1Y3PT9


We will have some merging to do. However these are the "clean" ones. The next ones are just ridiculous and should not exist in the first place. 

In [52]:
from collections import Counter
print(Counter(duplicated["ticker"].values.tolist()).most_common(5))
duplicated[duplicated["ticker"] == "DGICA"].head(5)
# ???

[('CMS', 217), ('PRE', 213), ('EP', 168), ('IBN', 31), ('CRESY', 24)]


,ticker,name,active,start_date,end_date,type,cik,composite_figi
4027,DGICA,Donegal Group Inc,False,2019-06-03,2022-08-25,CS,800457.0,BBG000JQJC22
4028,DGICA,Donegal Group Inc,False,2022-08-26,2022-08-26,CS,948046.0,BBG000JQJC22
4029,DGICA,Donegal Group Inc,False,2022-08-29,2022-09-02,CS,800457.0,BBG000JQJC22
4030,DGICA,Donegal Group Inc,False,2022-09-06,2022-09-06,CS,948046.0,BBG000JQJC22
4031,DGICA,Donegal Group Inc,False,2022-09-07,2022-09-07,CS,800457.0,BBG000JQJC22


In [53]:
duplicated[duplicated["ticker"] == "DISCK"].head(5)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
4104,DISCK,"Discovery, Inc. Series C Common Stock",False,2019-06-03,2022-02-07,CS,1024333.0,
4105,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-02-08,2022-02-08,CS,1024333.0,BBG000VMWHH5
4106,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-02-09,2022-03-02,CS,1024333.0,
4107,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-03-03,2022-03-03,CS,1024333.0,BBG000VMWHH5
4108,DISCK,"Discovery, Inc. Series C Common Stock",False,2022-03-04,2022-04-01,CS,1024333.0,


On average there are 3 duplicates for duplicated tickers. When we take a look it seems that it happens a lot that the name/cik/composite_figi gets changed, even though it is the same company and ticker. For example for ZWS the name is "Zurn Water Solutions Corporation" on 2022-07-01 but on the next trading day (4th July was a stock holiday) the name changes to "Zurn Elkay Water Solutions Corporation". 

# 3.5 Merging duplicates
The most straightforward way to merge these duplicates is to see for every duplicate whether the the end_date (1st occurence) and start_date (2nd occurence) are consecutive *trading days*. Now you understand why we made the functions in notebook 2.

In [54]:
tickers_v1.head(3)

,ticker,name,active,start_date,end_date,type,cik,composite_figi
0,A,Agilent Technologies,False,2019-06-03,2019-08-16,CS,1090872.0,BBG000C2V3D6
1,A,Agilent Technologies Inc.,False,2019-08-19,2022-02-07,CS,1090872.0,BBG000C2V3D6
2,A,Agilent Technologies Inc.,False,2022-02-08,2022-02-08,CS,1090872.0,BBG000BWQYZ5


First we need to get the duplicates.

In [55]:
tickers_v1 = get_tickers(1)
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
active = duplicated[duplicated['active']]
active[active["ticker"].duplicated(keep=False)]

,ticker,name,active,start_date,end_date,type,cik,composite_figi


In [56]:
###
tickers_v1 = get_tickers(1)
market_days = get_market_dates()

duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)] # Get all duplicated, *including* the original

# Step 1: Get the indices of the rows that should be merged.
indices_duplicated = [] # looks like [['A', {1, 2, 3}], ['A', {4, 5}], ['B', {10, 11, 12, 13}]]
prev_index_and_row = None
prev_is_duplicate_and_back_to_back = False

for index, row in duplicated.iterrows():
    # Get attributes of previous ticker
    if prev_index_and_row is not None:
        prev_index = prev_index_and_row[0]
        prev_row = prev_index_and_row[1]
        prev_ticker = prev_row["ticker"]
        prev_name = prev_row["name"]
        prev_start_date = prev_row["start_date"]
        prev_end_date = prev_row["end_date"]
        prev_cik = prev_row["cik"]
        prev_figi = prev_row["composite_figi"]

    # Get attributes of current ticker
    current_index = index
    current_row = row
    current_ticker = current_row["ticker"]
    current_name = current_row["name"]
    current_start_date = current_row["start_date"]
    current_end_date = current_row["end_date"]
    current_cik = current_row["cik"]
    current_figi = current_row["composite_figi"]
    
    # Skip first index
    if prev_index_and_row is None:
        pass
    # Check if ticker duplicated and back-to-back
    elif prev_ticker == current_ticker and market_days[market_days.index(prev_end_date) + 1] == current_start_date:
        # If the previous was NOT duplicated/back-to-back, we need to create a new entry for the stock
        if prev_is_duplicate_and_back_to_back == False:
            indices_duplicated.append([current_ticker, {prev_index, current_index}])
        # Else the stock already exists in the list. Then simply append the indices.
        else:
            indices_duplicated[-1][-1].add(prev_index)
            indices_duplicated[-1][-1].add(current_index)
        
        # Update flag
        prev_is_duplicate_and_back_to_back = True
    else:
        prev_is_duplicate_and_back_to_back = False

    # Update prev_index_and_row for next iteration
    prev_index_and_row = (current_index, row)

In [57]:
print(len(indices_duplicated))
print(indices_duplicated[:3])

3033
[['A', {0, 1, 2, 3}], ['AADI', {12, 13}], ['AAGR', {14, 15}]]


Very rarely, it happens that the same ticker, but not the same company, has duplicates. E.g. if in our ticker list the first 5 rows is the ticker AAA, but the start and end dates are (1, 2), (2, 3), (3, 4), (9, 10), (10, 11), this means that these are two different companies. Then indices_duplicated contains ['AAA', {1, 2, 3, 4}] and ['AAA, {10, 11}]. If it *is* the same company, it managed to get delisted to OTC and revive to get their listing back. Or something is wrong with Polygons data.

In [ ]:
# See which tickers are duplicated in indices_duplicated.
tickers_duplicates = set() 
for ticker, indices in indices_duplicated:
    if ticker in tickers_duplicates:
        print(ticker)
    tickers_duplicates.add(ticker)

Now that we have a list of indices of the duplicated tickers, we can merge them together. We do this by looping over <code>indices_duplicated</code> and then changing all duplicated rows to get the correct values. Then we remove the duplicates.

In [59]:
###

# Step 2: Merge duplicated in tickers_all
"""
Which value is assigned:
    name: last
    active: last
    start_date: first
    end_date: last
    type: last (but does not matter as it is always CS or ADRC)
    cik: last value that is not NaN
    compositite_figi: last value that is not NaN
"""
for ticker, indices in indices_duplicated:
    # CAUTION: Make sure that indices is sorted! Else it can happen that end_date is before start_date. I only found this out later. Moral: Always do sanity checks.
    indices = sorted(list(indices))
    ticker_data_in_tickers_v1 = tickers_v1.iloc[indices, :]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("name")] = ticker_data_in_tickers_v1["name"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("active")] = ticker_data_in_tickers_v1["active"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("start_date")] = ticker_data_in_tickers_v1["start_date"].values[0]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("end_date")] = ticker_data_in_tickers_v1["end_date"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("type")] = ticker_data_in_tickers_v1["type"].values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("cik")] = ticker_data_in_tickers_v1["cik"].ffill().values[-1]
    tickers_v1.iloc[indices, tickers_v1.columns.get_loc("composite_figi")] = ticker_data_in_tickers_v1["composite_figi"].ffill().values[-1]

tickers_v1 = tickers_v1.drop_duplicates().reset_index(drop=True)

In [60]:
len(tickers_v1)

9766

Now only a fraction of the original duplicated tickers remain.

In [61]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))
pd.set_option('display.max_rows', None)

duplicated.head(5)

602


,ticker,name,active,start_date,end_date,type,cik,composite_figi
3,AAC,"AAC Holdings, Inc.",False,2019-06-03,2019-10-25,CS,1606180.0,BBG00K1Y3PT9
4,AAC,Ares Acquisition Corporation,False,2021-03-25,2023-11-06,CS,1829432.0,
26,AAXN,"Axon Enterprise, Inc. Common Stock",False,2019-06-03,2021-01-25,CS,1069183.0,BBG000BHJWG1
27,AAXN,"Axon Enterprise, Inc. Common Stock",True,2023-09-08,2024-03-01,CS,1069183.0,BBG000BHJWG1
41,ABIL,Ability Inc Ordinary Shares,False,2019-06-03,2019-12-26,CS,1652866.0,


We can see that some are the same company but not back-to-back. Some are different companies or went OTC and back, these are correct. However, for a lot of stocks, the first occurence only trades for a few days. That makes no sense. If you try to download data for these dates, you will see that there exists none.

So some stocks have a 'ghost' day just before their IPO. E.g. YGF was IPO'd on 2023-03-28. But on 2023-03-24 had a entry with start_date and end_date of just one day. This is the same with VCIG, which had 2 'ghost' days on 2023-03-22 and 2023-04-06. For SXTP, the ghost days were actually two. Investigating the stocks that only have 1 day in our ticker list also shows funds (that are NOT common stocks!).

Nevertheless, if start_date is equal to end_date, it's always unusable and something is wrong. So we will first remove all tickers that only exist for one day.

In [62]:
ghost_days = tickers_v1[(tickers_v1["end_date"] - tickers_v1["start_date"]) == timedelta(days=0)]
print(len(ghost_days))
ghost_days.head(5)

552


,ticker,name,active,start_date,end_date,type,cik,composite_figi
98,ACLL,"ACell, Inc. Common Stock",False,2020-07-17,2020-07-17,CS,,
110,ACP,abrdn Income Credit Strategies Fund,False,2022-08-22,2022-08-22,CS,1503290.0,BBG0017VSC04
120,ACT,"Enact Holdings, Inc. Common Stock",False,2021-05-13,2021-05-13,CS,1823529.0,BBG00WSNP4R3
128,ACV,Virtus Diversified Income & Convertible Fund,False,2022-08-22,2022-08-22,CS,1636289.0,BBG008HMBD22
137,ADCT,ADC Therapeutics SA,False,2019-10-02,2019-10-02,CS,1771910.0,


In [63]:
###

# Only keep tickers that have >1 day history. But if they were just listed, keep them anyways.
tickers_v1 = tickers_v1[((tickers_v1["end_date"] - tickers_v1["start_date"]) > timedelta(days=0)) | 
                        (tickers_v1["end_date"] == market_days[market_days.index(END_DATE) - 1])]
len(tickers_v1)

9216

The remaining duplicates are:

In [64]:
duplicated = tickers_v1[tickers_v1["ticker"].duplicated(keep=False)]
print(len(duplicated["ticker"].unique()))

duplicated.head(5)

171


,ticker,name,active,start_date,end_date,type,cik,composite_figi
3,AAC,"AAC Holdings, Inc.",False,2019-06-03,2019-10-25,CS,1606180.0,BBG00K1Y3PT9
4,AAC,Ares Acquisition Corporation,False,2021-03-25,2023-11-06,CS,1829432.0,
26,AAXN,"Axon Enterprise, Inc. Common Stock",False,2019-06-03,2021-01-25,CS,1069183.0,BBG000BHJWG1
27,AAXN,"Axon Enterprise, Inc. Common Stock",True,2023-09-08,2024-03-01,CS,1069183.0,BBG000BHJWG1
41,ABIL,Ability Inc Ordinary Shares,False,2019-06-03,2019-12-26,CS,1652866.0,


We need to keep in mind that the <code>start_date</code> and <code>end_date</code> may not be the start/end dates of the data. To determine the data dates, we need to loop through the ticker list and see whether the data exists. 

However, after we have downloaded our data, we can just infer it. So we will postpone this to avoid doing it twice.

# 3.6 Removing incorrect classes
There are still some weird or incorrect stock classes that we have to remove. These were found by just looking through the ticker list.

These are:
- Funds
- Preferred stock/bonds
- A "w" appended to the stock ticker
- "Ex-distribution" or "When-issued" conditions

In [65]:
###
funds = tickers_v1[tickers_v1['name'].apply(lambda s: "Fund" in s.split())]
print(len(funds))
funds.head(3)

141


,ticker,name,active,start_date,end_date,type,cik,composite_figi
127,ACV,AllianzGI Div Inc & Convert Fund,False,2019-06-03,2019-08-14,CS,1636289.0,
176,ADX,Adams Diversified Equity Fund,False,2019-06-03,2019-08-09,CS,2230.0,BBG000BB8MR6
234,AFT,Apollo Senior Floating Rate Fund Inc.,False,2019-06-03,2019-09-05,CS,1502573.0,BBG00174L007


In [66]:
###
notes = tickers_v1[tickers_v1['name'].str.contains('%')]
print(len(notes))
notes.head(3)

55


,ticker,name,active,start_date,end_date,type,cik,composite_figi
45,ABLLL,"Abacus Life, Inc. 9.875% Fixed Rate Senior Not...",True,2023-11-28,2024-03-01,CS,,
83,ACGLN,"Arch Capital Group Ltd. Depositary Shares, eac...",True,2023-05-08,2024-03-01,CS,947484.0,
263,AGNCL,AGNC Investment Corp. Depositary Shares Each R...,True,2023-05-08,2024-03-01,CS,1423689.0,


In [67]:
###
tickers_w = tickers_v1[tickers_v1["ticker"].str.contains("w")]
print(len(tickers_w))
tickers_w.head(3)

114


,ticker,name,active,start_date,end_date,type,cik,composite_figi
16,AANw,"The Aaron''s Company, Inc.",False,2020-11-25,2020-11-30,CS,1821393.0,BBG00WCNDCZ6
167,ADSw,Alliance Data Systems Corporation,False,2021-11-01,2021-11-05,CS,1101215.0,
309,AIRCw,Apartment Income REIT Corp.,False,2020-12-03,2020-12-14,CS,1820877.0,BBG00XK3WVD0


In [68]:
###
when_issued_or_ex_distr = tickers_v1[tickers_v1['name'].apply(lambda s: ("When" in s.split()) or ("Issued" in s.split()) or ("When-Issued" in s.split()) or ("Ex-Distribution" in s.split())  )  ]
print(len(when_issued_or_ex_distr))
when_issued_or_ex_distr.head(3)

68


,ticker,name,active,start_date,end_date,type,cik,composite_figi
141,ADEAV,Adeia Inc. Common Stock Ex-distribution When I...,False,2022-09-20,2022-09-30,CS,,BBG019KN8702
379,ALKSV,Alkermes plc Ordinary Shares Ex-distribution W...,False,2023-11-03,2023-11-15,CS,1520262.0,
552,AOUTV,"American Outdoor Brands, Inc. Common Stock Whe...",False,2020-08-10,2020-08-24,CS,1808997.0,BBG00QV8FS02


In [69]:
###
wd_suffix = tickers_v1[tickers_v1["ticker"].str.contains("\.WD")]
print(len(wd_suffix))
wd_suffix.head(3)

7


,ticker,name,active,start_date,end_date,type,cik,composite_figi
2433,DD.WD,"DuPont de Nemours, Inc",False,2021-02-02,2021-02-03,CS,1666700.0,BBG00BN961G4
2502,DHR.WD,Danaher Corporation When Distributed,False,2019-12-16,2019-12-18,CS,313616.0,BBG000BH3JF8
2746,ECL.WD,Ecolab Inc.,False,2020-06-03,2020-06-05,CS,31462.0,BBG000BHKYH4


In [70]:
###
indices_to_remove = funds.index.union(notes.index).union(tickers_w.index).union(when_issued_or_ex_distr.index).union(wd_suffix.index)

In [71]:
###
print(len(tickers_v1))
tickers_v1 = tickers_v1.drop(index=indices_to_remove)
print(len(tickers_v1))

9216
8832


Apparently this is not enough. There are still a lot of misclassified funds/preferred stocks. For example the ticker ARDC (ARES DYNAMIC CREDIT ALLOCATION FUND, INC.) is considered a common stock on 2019-04-02. However on 2019-04-01 it is considered a fund. It seems that Polygon uses the filings to determine the classification, and that common stocks are the default class. Then sometimes the filings are incomplete and Polygon classifies it as a common stock.

In [72]:
funds = tickers_v1[tickers_v1["name"].str.lower().str.contains("fund") | tickers_v1["name"].str.lower().str.contains("strategies")]
print(len(funds))
funds.head(5)

47


,ticker,name,active,start_date,end_date,type,cik,composite_figi
109,ACP,Aberdeen Income Crd Strategies,False,2019-06-03,2019-08-09,CS,1503290.0,BBG0017VSC04
632,ARDC,"ARES DYNAMIC CREDIT ALLOCATION FUND, INC.",False,2019-06-03,2019-09-05,CS,1515324.0,BBG001LJH325
702,ASFI,Asta Funding Inc,False,2019-06-03,2020-09-29,CS,1001258.0,BBG00K9VT498
1093,BGB,BLACKSTONE / GSO STRATEGIC CREDIT FUND,False,2019-06-03,2019-09-09,CS,1546429.0,BBG002W5R785
2416,DCF,BNY Mellon Alcentra Global Credit Income 2024 ...,False,2019-06-03,2019-09-09,CS,1627854.0,BBG00J2DVN20


In [73]:
client.get_ticker_details(ticker="BGIO", date = "2019-09-13").type

'FUND'

In [74]:
client.get_ticker_details(ticker="ACP", date = "2019-04-02").type

'CS'

In [75]:
client.get_ticker_details(ticker="ACP", date = "2019-04-01").type

'FUND'

We will solve this issue by looking at the classification before or after the start/end date. If it is not a common stock, we will delete it from the ticker list. This takes quite some time to run. However most misclassifications seem to be in 2019. So realistically I would only do this once.

In [76]:
###
tickers_v1["ID"] = tickers_v1["ticker"] + '-' + tickers_v1["start_date"].astype(str)

In [ ]:
### (OPTIONAL)
IDs_to_remove = []
for index, row in tickers_v1.copy().iterrows():
    try:
        # The get_ticker_details is extremely slow.
        type_before_start = client.get_ticker_details(ticker=row['ticker'], date = last_trading_date_before(row['start_date']) ).type
    except Exception as e:
        # If there is no information. we will keep it.
        type_before_start = 'CS'

    try: 
        type_after_end = client.get_ticker_details(ticker=row['ticker'], date = first_trading_date_after(row['end_date']) ).type
    except Exception as e:
        # If there is no information. we will keep it.
        type_after_end = 'CS'

    # Apparently an ordinary share is the same as a common stock... Also do not forget ADRCs!
    if type_before_start not in ['CS', 'ADRC', 'OS', None] or type_after_end not in ['CS', 'ADRC', 'OS', None]:
        IDs_to_remove.append(row['ID'])
        print(row['ID'])

The amount of incorrect classified common stock is unacceptable:

In [83]:
### (OPTIONAL)
incorrect_classified = tickers_v1[tickers_v1['ID'].isin(IDs_to_remove)]
print(len(incorrect_classified))
incorrect_classified.to_csv("../data/incorrect_classified.csv")
incorrect_classified.tail(3)[['ticker', 'name', 'active', 'start_date', 'end_date']]

243


,ticker,name,active,start_date,end_date
9074,UTF,"Cohen & Steers Infrastructure Fund, Inc.",False,2019-06-03,2019-09-10
9549,WTMAR,Welsbach Technology Metals Acquisition Corp. o...,False,2022-01-21,2022-08-29
9681,YSACW,Yellowstone Acquisition Company Warrants to pu...,False,2020-12-08,2021-02-09


In [84]:
###
to_remove = pd.read_csv("../data/incorrect_classified.csv", index_col=0)
IDs_to_remove = list(to_remove.index)

print(len(tickers_v1))
tickers_v1 = tickers_v1[~tickers_v1['ID'].isin(IDs_to_remove)]
tickers_v1.reset_index(inplace=True, drop=True)
print(len(tickers_v1))

8832
8589


Finally, we save the merged and cleaned ticker list to <code>tickers_v2.csv</code>.

In [85]:
###
print(f"Total tickers: {len(tickers_v1)}")
print(f"Unique tickers: {len(tickers_v1['ticker'].unique())}")

tickers_v1 = tickers_v1.reset_index(drop=True)
tickers_v1 = tickers_v1[["ID", "ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]]
tickers_v1.to_csv("../data/tickers_v2.csv")

Total tickers: 8589
Unique tickers: 8434


: 

# 3.7 Updates
1. Run the first cell below to create a backup of <code>tickers_v3</code>. Later when we download data, we will compare the old to the new ticker list to determine which stocks and dates to update, instead of downloading everything.
1. Update END_DATE.
2. Run the 3 ### cells in 3.1. This updates the folder of ticker lists. It takes around 7 seconds per day.
3. Run the cell below to update <code>tickers_v1</code>. Instead of merging all ticker lists, only the new ones are merged.
4. Run the cells with ### in 3.5 and 3.6 to update <code>tickers_v2</code>. We skip checking if a stock is a common stock is not because it is very time consuming. Instead we remove what we had already removed. If you do not want to skip it, run the two '### (OPTIONAL)' rows. However most incorrect classifications were in 2019.

In [ ]:
tickers_v3 = get_tickers(3)
tickers_v3.to_csv("../data/tickers_v3_old.csv")

In [ ]:
from tickers import get_tickers

tickers_v1 = get_tickers(1, cik_as_float=False)
current_end_date = tickers_v1['end_date'].max()

market_days = get_market_dates()
first_trading_date_after_current_end_date = first_trading_date_after_equal(current_end_date + timedelta(days=1))
last_trading_date_before_end_date = last_trading_date_before_equal(END_DATE)

for day in market_days:
    if day == current_end_date:
        our_tickers = tickers_v1
        our_tickers.loc[our_tickers['active'], 'end_date'] = np.NaN

    elif day >= first_trading_date_after_current_end_date and day <= END_DATE:
        # Get new ticker list to update ours
        tickers_day_i = pd.read_csv(
            POLYGON_DATA_PATH + f"raw/tickers/{day.isoformat()}.csv",
            index_col=0,
            keep_default_na=False,
            na_values=['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null']
        )
        tickers_day_i = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]]
        tickers_day_i = tickers_day_i[tickers_day_i["active"] == True]
        tickers_day_i.reset_index(inplace=True, drop=True)

        # Preliminary check: no duplicates
        if our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        if tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].duplicated().all():
            raise Exception("There are duplicates!")

        # DELISTINGS
        indicator_delisted = our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], how='left', indicator=True)

        indicator_delisted['_merge'] = np.where(our_tickers["active"], indicator_delisted['_merge'], "both") # ERROR FIX: If in our ticker list we have already set it inactive, it should not be added to the list of delisted stocks again. By setting _merge to "both" we skip the already inactive stocks.

        indicator_delisted = indicator_delisted["_merge"] # Only get the indicator
        delisted_tickers = our_tickers[indicator_delisted == "left_only"]

        # NEW LISTINGS
        indicator_new = tickers_day_i[["ticker", "name", "active", "cik", "composite_figi", "type"]].merge(our_tickers[["ticker", "name", "active", "cik", "composite_figi", "type"]], on=["ticker", "name", "active", "cik", "composite_figi", "type"], 
                        how='left', indicator=True)
        indicator_new = indicator_new["_merge"]
        new_tickers = tickers_day_i[indicator_new == "left_only"]

        # PROCESS DELISTINGS
        previous_day = market_days[market_days.index(day) - 1] # Getting previous trading day
        our_tickers.loc[indicator_delisted == "left_only", "end_date"] = previous_day
        our_tickers.loc[indicator_delisted == "left_only", "active"] = False
        
        # PROCESS NEW LISTINGS
        our_tickers = pd.concat([our_tickers, new_tickers])

        our_tickers.reset_index(inplace=True, drop=True)
        our_tickers['start_date'].fillna(value=day, inplace=True)
        
        # Final checks
        if our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().values.any():
            #null_data = our_tickers[our_tickers[["ticker", "name", "active", "type", "start_date"]].isnull().any(axis=1)]
            raise Exception("There are missing values.")
        
        print(f'{day.isoformat()}: Amount of stocks {len(our_tickers)}')
        
        # Finalize
        if day == last_trading_date_before_end_date:
            our_tickers["end_date"].fillna(value=last_trading_date_before_end_date, inplace=True)
            our_tickers = our_tickers.sort_values(by=["ticker", "end_date"]).reset_index(drop=True)
            our_tickers[["ticker", "name", "active", "start_date", "end_date", "type", "cik", "composite_figi"]].to_csv("../data/tickers_v1.csv")

2023-09-05: Amount of stocks 14794
2023-09-06: Amount of stocks 14799
2023-09-07: Amount of stocks 14802
